#### Atividade Busca Semântica

Nesta atividade você deve aplicar os conhecimentos sobre **word embeddings e tokenização** para criar um mecanismos de busca semântica. Será disponibilizado um conjunto de dados que possui perguntas médicas, em inglês. Esse conjunto de dados possui as perguntas e também as respostas.

Você deve **gerar os vetores das perguntas do conjunto de dados**, e permitir que o "usuário" envie a sua pergunta. Você também deve gerar o vetor da **pergunta do usuário** e com isso buscar a resposta ideal para o usuário.

Portanto, no seu script deve ser possível escrever um texto "pergunta" e deve ser retornado a resposta adequada, isto é, a resposta associada a pergunta mais **similar** no conjunto de dados.

Para isso utilize o pandas e os packages do huggingface


In [28]:
### Download do conjunto de dados

import kagglehub

# Download latest version
path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")

print("Path to dataset files:", path)

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/pythonafroz/medquad-medical-question-answer-for-ai-research/versions/1


In [29]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers.tokenization_utils_base import BatchEncoding

In [30]:
## Amostra do conjunto

df = pd.read_csv(f"{path}/medquad.csv")
df_amostra = df.sample(100)

In [31]:
df.head

<bound method NDFrame.head of                                                 question  \
0                               What is (are) Glaucoma ?   
1                                 What causes Glaucoma ?   
2                    What are the symptoms of Glaucoma ?   
3                 What are the treatments for Glaucoma ?   
4                               What is (are) Glaucoma ?   
...                                                  ...   
16407  What is (are) Diabetic Neuropathies: The Nerve...   
16408  How to prevent Diabetic Neuropathies: The Nerv...   
16409  How to diagnose Diabetic Neuropathies: The Ner...   
16410  What are the treatments for Diabetic Neuropath...   
16411  What to do for Diabetic Neuropathies: The Nerv...   

                                                  answer           source  \
0      Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   
1      Nearly 2.7 million people have glaucoma, a lea...  NIHSeniorHealth   
2      Symptoms of

In [32]:
# Modelo para a língua inglesa

nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo)

In [33]:
## Funções para: 1) Obter os tokens; 2) Obter os embeddings

def get_tokens(pergunta: str) -> str:
    return tokenizer(pergunta, return_tensors="pt")

def get_vetores(tokens_pergunta: BatchEncoding) -> Tensor:

    with torch.no_grad():
        outputs = model(**tokens_pergunta)
        embeddings = outputs.last_hidden_state

    return embeddings

In [34]:
## Novas colunas para os tokens e para os embeddings

df_amostra['tokens'] = df_amostra['question'].apply(lambda x: get_tokens(x))
df_amostra['vetores'] = df_amostra['tokens'].apply(lambda x: get_vetores(x))

In [35]:
def get_cls_embedding(vetores: torch.Tensor) -> torch.Tensor:
    return vetores[0,0,:]

df_amostra['cls_embedding'] = df_amostra['vetores'].apply(get_cls_embedding)

In [36]:
import torch.nn.functional as F

def buscar_resposta(pergunta_usuario: str, df: pd.DataFrame):
    tokens_usuario = get_tokens(pergunta_usuario)
    vetores_usuario = get_vetores(tokens_usuario)
    emb_usuario = get_cls_embedding(vetores_usuario)

    embeddings_dataset = torch.stack(df['cls_embedding'].tolist())

    similaridades = F.cosine_similarity(emb_usuario.unsqueeze(0), embeddings_dataset)

    idx_max = torch.argmax(similaridades).item()

    pergunta_mais_similar = df.iloc[idx_max]['question']
    resposta_ideal = df.iloc[idx_max]['answer']
    score = similaridades[idx_max].item()

    return pergunta_mais_similar, resposta_ideal, score

In [37]:
pergunta_usuario = input('Digite sua pergunta em inglês: ')
pergunta_similar, resposta, score = buscar_resposta(pergunta_usuario, df_amostra)
print(f'Pergunta mais similar encontrada:\n{pergunta_similar}\n')
print(f'Resposta sugerida:\n{resposta}\n')
print(f'Similaridade: {score:.4f}')

Pergunta mais similar encontrada:
What is (are) Mondini dysplasia ?

Resposta sugerida:
Mondini dysplasia is a type of inner ear malformation that is present at birth (congenital). Individuals with Mondini dysplasia have one and a half coils of the cochlea instead of the normal two coils. It may occur in one ear (unilateral) or both ears (bilateral) and can cause varying degrees of sensorineural hearing loss, although most individuals have profound hearing loss. The condition can also predispose affected individuals to recurrent meningitis. It is caused by disruption in the embryonic development of the inner ear during the seventh week of gestation. The condition may be isolated (occurring with no other conditions or malformations) or may occur with other ear malformations or a number of syndromes. Treatment options may include surgical repair of the defect to prevent recurrent meningitis; amplification aids for those with residual hearing; and cochlear implantation.

Similaridade: 0.9